In [ ]:
import pandas as pd

df = pd.read_csv('./data_by_year.csv')

In [ ]:
df = pd.DataFrame(df)
df = df[['popularity', 'year']]

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df[['popularity']],df[['year']], alpha = 0.6)

In [ ]:
import sys
!{sys.executable} -m pip install statsmodels

import statsmodels.api as sm

y = df[['popularity']]
x = df[['year']]
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())


In [ ]:
from sklearn import linear_model

x = df[['year']].values.tolist()
y = df[['popularity']].values.tolist()

fig, ax = plt.subplots(1, 1)
ax.scatter(x, y, s=10)

regr = linear_model.LinearRegression()
X = x
Y = y
regr.fit(X, Y)  # use fit method
r_sqr = regr.score(X, Y)
betas = regr.coef_  # m
y_int = regr.intercept_  # b

# Visualize: actual vs. predicted income (from model)
y_hat = betas[0] * x + y_int
plt.plot(x, y_hat, color='orange')  # y_hat, income

In [ ]:
df = pd.read_csv('./full_music_data.csv')

df = pd.DataFrame(df)
df = df[['popularity', 'artist_names', 'song_title (censored)', 'year']]

In [ ]:
df

In [ ]:
year_list = df['year'].values.tolist()

std_c = 0.017/0.7045 + 33.210/1360.9905

# simulation_func = 0.7045x - 1360.9905

expected_pop = []

for i in year_list:
    simul_result = 0.7045*i - 1360.9905
    simul_interval = [simul_result, simul_result * std_c]
    expected_pop.append(simul_interval)

In [ ]:
expected_pop

In [ ]:
df['simulated_popularity'] = expected_pop

In [ ]:
import random

excep_list = []

for i in range(len(df)):
    lst = df.at[i,'simulated_popularity']
    count = 0
    valid = 0
    for t in range(100):
        val = random.gauss(lst[0], lst[1])
        actual_val = df.at[i,'popularity']
        try:
            excep = -val + actual_val
            count += excep
            valid += 1
        except:
            excep = 'nan'
    excep_list.append(count/valid)  

In [ ]:
df['sig'] = excep_list

In [ ]:
df

In [ ]:
import numpy as np

df = df.sort_values(by = ['sig'], ascending = False)
df.replace([np.inf, -np.inf], np.nan)
df.dropna(inplace=True)

df = df[df['sig'] != np.inf]
df = df[df['sig'] != -np.inf]


In [ ]:
df.head(40)

In [ ]:
df[df['popularity'] > 10].head(100)

In [ ]:
df.to_csv('./reverseTrend.csv')

In [ ]:
df = pd.read_csv('data_by_artist.csv')
df2 = pd.read_csv('full_music_data.csv')
df3 = df2
df3 = df3.drop_duplicates(subset='artist_names', keep='first', inplace=False)
df3
print('===')
df2

In [ ]:
import re
import scipy.stats as st

all_artist = df3['artist_names'].values.tolist()
expand_artist = []
for i in all_artist:
    lst = eval(i)
    for j in lst:
        expand_artist.append(j)
re_expand_artist = sorted(set(expand_artist),key=expand_artist.index)

year_coll = []
for i in re_expand_artist:
    sing_coll = []
    for j in range(len(df2)):
        if i in df2.at[j,'artist_names']:
            sing_coll.append(df2.at[j, 'year'])
    sing_coll = np.array(sing_coll)

    loc=np.mean(sing_coll)
    scale=st.sem(sing_coll)
        
    if scale == 0 or len(sing_coll) == 1:
        conf_interval = [int(loc), int(loc)]
    else:
        conf_interval = list(st.norm.interval(alpha=0.90, loc=np.mean(sing_coll), scale=st.sem(sing_coll)))
    print(i)
    print(sing_coll)
    print(conf_interval)
    year_coll.append(conf_interval)
    
#     print(i)
#     print(sing_coll)

In [ ]:
print(re_expand_artist[0], year_coll[0])

In [ ]:
year_intv_coll = []

for the_year in year_coll:
    std_c = 0.017/0.7045 + 33.210/1360.9905
    lower_exp = 0.7045*the_year[0] - 1360.9905
    upper_exp = 0.7045*the_year[1] - 1360.9905
    expected_pop = [lower_exp, upper_exp]
    year_intv_coll.append(expected_pop)
    
        

In [ ]:
year_intv_coll[0]

In [ ]:
df = pd.DataFrame()
df['artist'] = re_expand_artist
df['expected_popularity'] = year_intv_coll
df['average_working_year'] = year_coll

In [ ]:
df

In [ ]:
#data by artist
df4 = pd.read_csv('data_by_artist.csv')
df['actual_popularity'] = np.nan
loc = 0
for i in df['artist']:
    print(i, df.at[loc, 'artist'])
    for k in range(len(df4)):
        if df4.at[k,'artist_name'] == i:
            df.at[loc, 'actual_popularity'] = df4.at[k,'popularity']
    loc += 1
            

In [ ]:
df.head(50)

In [ ]:
df.to_csv('pop_exp_sim.csv')

In [ ]:
import pandas as pd

pdd = pd.read_csv('pop_exp_sim.csv')
pdd['mean'] = ''
pdd

In [ ]:
count = 0
for i in pdd['expected_popularity']:
    print(i)
    mean = (eval(i)[0] + eval(i)[1])/2
    pdd.at[count,'mean'] = mean
    count += 1

In [ ]:
pdd

In [ ]:
pdd

In [ ]:
for i in range(len(pdd)):
    pdd.at[i,'diff'] = pdd.at[i,'actual_popularity'] - pdd.at[i,'mean']

In [ ]:
pdd

In [ ]:
pdd = pdd.sort_values(by = ['diff'], ascending = False)
pdd.to_csv('random.csv')

In [ ]:
pddd = pd.read_csv('betweeness.csv', encoding='unicode_escape')
pddd['working year'] = ''
pddd

In [ ]:
count = 0
for i in pddd['Artist']:
    for j in range(len(pdd)):
        if pdd.at[j, 'artist'] == i:
            pddd.at[count, 'working year'] = pdd.at[j, 'average_working_year']
    print(count)
    count += 1

In [ ]:
for i in range(0, len(pddd)):
    try: 
        kk = pddd.at[i, 'working year']
        pddd.at[i, 'working year'] = kk * (-0.02643606442739991) + 54.58391785656113
    except SyntaxError: 
        pddd.at[i, 'working year'] = np.nan

In [ ]:
pddd

In [ ]:
import math
for i in range(0, len(pddd)):
    try: 
        kk = pddd.at[i, 'working year']
        pddd.at[i, 'working year'] = math.log(kk)
    except SyntaxError: 
        pddd.at[i, 'working year'] = np.nan

In [ ]:
pddd

In [ ]:
pddd['updated_1'] = ''
for i in range(len(pddd)):
    pddd.at[i,'updated_1'] = pddd.at[i,'betweeness'] * pddd.at[i,'working year']

In [ ]:
pddd.sort_values(by = ['updated_1'], ascending = False)

In [ ]:
pddd = pd.read_csv('updated_1.csv')
pddd.dropna()

In [ ]:
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))

pf = pddd[['updated_1']].apply(max_min_scaler)

In [ ]:
pddd['updated_1'] = pf
pddd.to_csv('updated_1.csv')

In [ ]:
import pandas as pd

influ_file = pd.read_csv('influence.csv', encoding='unicode_escape')
updated_1 = pd.read_csv('updated_1.csv', encoding='unicode_escape')

In [ ]:
import numpy as np

max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))

pf = influ_file[['Influence Index']].apply(max_min_scaler)
influ_file['norm_index'] = pf

In [ ]:
influ_file

In [ ]:
updated_1

In [ ]:
for i in range(len(updated_1)):
    the_artist = updated_1.at[i, 'Artist']
    for j in range(len(influ_file)):
        if the_artist == influ_file.at[j, 'Artist']:
            updated_1.at[i, 'norm_influ_index'] = influ_file.at[j, 'norm_index']
            print(the_artist)
            

In [ ]:
updated_1

In [ ]:
import math

for i in range(len(updated_1)):
    updated_1.at[i,'updated_2'] = updated_1.at[i,'updated_1'] * math.log(updated_1.at[i,'norm_influ_index'] * 10000)

In [ ]:
updated_1.dropna()

In [ ]:
pf = updated_1[['updated_2']].apply(max_min_scaler)
updated_1['norm_updated_2'] = pf

In [ ]:
updated_1

In [ ]:
updated_1.drop(columns = ['updated_2'])
updated_1.dropna()

In [ ]:
updated_1 = updated_1.sort_values(by = ['norm_updated_2'], ascending = False)

In [ ]:
updated_1.to_csv('updated_2.csv')